In [1]:
# common packages
import pandas as pd
import time

# # DL framework
import torch
from torchtext import data

from attractivedata import AttractiveData
from trainer import AttractiveTrainer

In [2]:
# torch.manual_seed(123)
# torch.cuda.manual_seed(123)
# torch.backends.cudnn.deterministic = True

## Load and prepare data

In [3]:
train_file = 'data/train.csv'
test_file = 'data/test.csv'
pretrained_file = 'glove.840B.300d'
config = {
    'max_seq': 40,
    'min_freq': 0,
    'batch_size': 64,
    'pretrained_file': pretrained_file
}


In [4]:
AttractiveData = AttractiveData(train_file, test_file, pretrained_file, config)

In [5]:
# for i, sentence in enumerate(AttractiveData.test_data):
#     if i == 3:
#         print(vars(AttractiveData.train_data[i]), vars(sentence))

In [6]:
len(AttractiveData.CATEGORIES_LABEL.vocab.freqs)

18

In [7]:
max_len = 0
a = AttractiveData.train_data
for i in range(len(a)):
    if len(a[i].Headline) >= max_len:
        max_len = len(a[i].Headline)
max_len

38

## Start to train

In [8]:
config['timestr'] = time.strftime("%Y%m%d-%H%M%S")
config['save_name'] = 'CNN_LSTM'
config['input_dim'] = len(AttractiveData.TEXT.vocab)
config['embedding_dim'] = 300
config['category_dim'] = len(AttractiveData.CATEGORIES_LABEL.vocab)
config['category_embedding_dim'] = 16
config['hidden_dim'] = 30
config['output_dim'] = 1
config['log_steps'] = 10
config['epochs'] = 100
config['lr'] = {
    'encoder': 1e-5,
    'embedding': 1e-5,
    'linear': 1e-5
}
config['num_layers'] = 1
config['kernel_size'] = 3
config['dropout'] = 0.1
config['train_len'] = AttractiveData.train_len
config['test_len'] = AttractiveData.test_len

pretrained_embeddings = AttractiveData.TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([12699, 300])


In [9]:
AttractiveTrainer = AttractiveTrainer(config, AttractiveData.device, AttractiveData.trainloader, pretrained_embeddings)

In [10]:
AttractiveTrainer.model, AttractiveTrainer.config['total_params'], AttractiveTrainer.config['total_learned_params']

(AttractiveNet(
   (embedding): AttractiveEmbedding(
     (token): TokenEmbedding(12699, 300, padding_idx=1)
   )
   (cnn1): Sequential(
     (0): Conv1d(300, 220, kernel_size=(3,), stride=(1,), padding=(1,))
     (1): ReLU()
   )
   (cnn2): Sequential(
     (0): Conv1d(220, 150, kernel_size=(3,), stride=(1,), padding=(1,))
     (1): ReLU()
   )
   (cnn3): Sequential(
     (0): Conv1d(150, 100, kernel_size=(3,), stride=(1,), padding=(1,))
     (1): ReLU()
   )
   (encoder): LSTM(100, 30, batch_first=True, dropout=0.1, bidirectional=True)
   (linear): Sequential(
     (0): Linear(in_features=120, out_features=30, bias=True)
     (1): ReLU()
     (2): Linear(in_features=30, out_features=1, bias=True)
   )
 ),
 4187511,
 377811)

In [11]:
AttractiveTrainer.train()

Epoch:   1%|          | 1/100 [00:00<00:23,  4.17it/s]
EP_train | avg_loss: 0.5321353903003767 |
====
Epoch:   2%|▏         | 2/100 [00:00<00:22,  4.31it/s]
EP_train | avg_loss: 0.531874101302203 |
====
Epoch:   3%|▎         | 3/100 [00:00<00:21,  4.44it/s]
EP_train | avg_loss: 0.5317850384057737 |
====
Epoch:   4%|▍         | 4/100 [00:00<00:21,  4.54it/s]
EP_train | avg_loss: 0.5317583897534539 |
====
Epoch:   5%|▌         | 5/100 [00:01<00:20,  4.58it/s]
EP_train | avg_loss: 0.5317216209336824 |
====
Epoch:   6%|▌         | 6/100 [00:01<00:20,  4.60it/s]
EP_train | avg_loss: 0.5316833795285693 |
====
Epoch:   7%|▋         | 7/100 [00:01<00:20,  4.62it/s]
EP_train | avg_loss: 0.5316589514414469 |
====
Epoch:   8%|▊         | 8/100 [00:01<00:19,  4.68it/s]
EP_train | avg_loss: 0.5316087479684867 |
====
Epoch:   9%|▉         | 9/100 [00:01<00:19,  4.74it/s]
EP_train | avg_loss: 0.5315781135185093 |
====
Epoch:  10%|█         | 10/100 [00:02<00:18,  4.76it/s]
EP_train | avg_loss: 0.5315

## for classification, not better

## Below is testing

In [12]:
from attractivenet import AttractiveNet
PATH = './model/CNN_LSTM_20201101-160258_0.3556.100'
# load_model = TransformerModel(config).to(AttractiveData.device)
load_model = AttractiveNet(config).to(AttractiveData.device)
load_model.load_state_dict(torch.load(PATH))
load_model.eval()

AttractiveNet(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(12699, 300, padding_idx=1)
  )
  (cnn1): Sequential(
    (0): Conv1d(300, 220, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
  )
  (cnn2): Sequential(
    (0): Conv1d(220, 150, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
  )
  (cnn3): Sequential(
    (0): Conv1d(150, 100, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
  )
  (encoder): LSTM(100, 30, batch_first=True, bidirectional=True)
  (linear): Sequential(
    (0): Linear(in_features=120, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=1, bias=True)
  )
)

In [16]:
def predict_attractive(sentence, category, phase):
    # if len(sentence) < config['max_seq']:
    #     sentence += ['0'] * (config['max_seq'] - len(sentence))
    # else:
    #     sentence = sentence[:config['max_seq']]

    indexed_sentence = [AttractiveData.TEXT.vocab.stoi[t] for t in sentence]
    indexed_category = [AttractiveData.CATEGORIES_LABEL.vocab.stoi[category]]
    tensor_sentence = torch.LongTensor(indexed_sentence).to(AttractiveData.device)
    tensor_category = torch.LongTensor(indexed_category).to(AttractiveData.device)

    tensor_sentence = tensor_sentence.unsqueeze(0)
    # print(tensor_sentence.shape)

    prediction = load_model(tensor_sentence, tensor_category, phase=phase)
    
    return prediction

In [17]:
# train mean = 3.15, test mean = 2.8
predict_list = []
with torch.no_grad():
    for i, sentence in enumerate(AttractiveData.test_data):
        prediction = predict_attractive(sentence.Headline, sentence.Category, 'test')
        predict_list.append(prediction.item())
        # predict_list.append(prediction.item())
AttractiveData.df_test['Label'] = predict_list
AttractiveData.df_test[['ID', 'Label']].to_csv(config['save_name'] + '.csv', index=False)

In [18]:
# train_category = list(AttractiveData.CATEGORIES_LABEL.vocab.freqs)
# test_category = list(AttractiveData.df_test['Category'].value_counts().keys())
# for each_test in test_category:
#     if each_test not in train_category:
#         print(each_test)
# print()
# for each_train in train_category:
#     if each_train not in test_category:
#         print(each_train)

## Below just for fun guess

In [19]:
import statistics
from sklearn.metrics import mean_squared_error

In [20]:
# train mean = 3.15, test mean = 2.8
train_list = []
for i, sentence in enumerate(AttractiveData.train_data):
    prediction = predict_attractive(sentence.Headline, sentence.Category, 'train')
    train_list.append(prediction.item())
    # train_list.append(prediction.item())
# print(train_list)
mean_squared_error(pd.read_csv('data/train.csv').sort_values(['ID']).Label.to_list(), train_list), statistics.mean(train_list), statistics.stdev(train_list)

(0.008360719004864103, 3.138601073155216, 0.7192296302016682)

In [21]:
train_list[0:5], pd.read_csv('data/train.csv').sort_values(['ID']).Label.to_list()[0:5]

([4.00521183013916,
  2.2720608711242676,
  4.3802032470703125,
  3.3262126445770264,
  3.997366428375244],
 [4.0, 2.333333333333333, 4.5, 3.333333333333333, 4.0])

In [22]:
a = AttractiveData.df_train['Label'].to_list()
statistics.mean(a), statistics.stdev(a)

(3.150408496732026, 0.729501519321601)

In [23]:
statistics.mean(predict_list), statistics.stdev(predict_list)

(2.626068512773724, 0.5738398423117412)

In [24]:
baseline_list = pd.read_csv('baseline.csv').sort_values(['ID']).Label.to_list()
mean_squared_error(baseline_list, predict_list), statistics.mean(baseline_list), statistics.stdev(baseline_list)

(0.26562163420502694, 2.7156126782757597, 0.29355123275379763)

In [14]:
# LSTM my best
# statistics.mean(predict_list), statistics.stdev(predict_list)

(2.8167915543795683, 0.14611407210842048)

In [21]:
a = pd.read_csv('LSTM_base.csv').Label.to_list()
mean_squared_error(baseline_list, a), statistics.mean(a), statistics.stdev(a)

(0.1347375515605904, 2.8379913731293533, 0.1903582104725371)

In [14]:
AttractiveData.TEXT.vocab.itos[2]

'the'